<a href="https://colab.research.google.com/github/tambean29/Bookstore/blob/main/ML_Project_CNN_BreastCancerClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 8.2 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras_tuner as kt

In [ ]:
# from tensorflow.keras.applications import EfficientNetB0
# from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, BatchNormalization

# def transfer_model_builder(hp):
#     base_model = EfficientNetB0(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3), weights='imagenet')
#     base_model.trainable = False

#     model = Sequential()
#     model.add(base_model)
#     model.add(GlobalAveragePooling2D())
#     model.add(BatchNormalization())

#     hp_dropout = hp.Float('dropout', 0.2, 0.5, step=0.1)
#     model.add(Dropout(hp_dropout))

#     hp_regularization = hp.Float('regularization',min_value=0.0,max_value=0.5,step=0.1)
#     hp_units = hp.Int('units', min_value=128, max_value=512, step=32)
#     model.add(Dense(hp_units, activation='relu',kernel_regularizer= keras.regularizers.l2(hp_regularization)))

#     model.add(Dense(1, activation='sigmoid'))

#     hp_learning_rate = hp.Choice('lr', [1e-2, 1e-3, 1e-4])
#     model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
#                   loss='binary_crossentropy',
#                   metrics=['accuracy'])

#     return model


In [ ]:
def cnn_builder(hp):

    model = Sequential()

    hp_filters1 = hp.Int('filters1',min_value=32,max_value=128,step=8)
    hp_kernelrow = hp.Int('kernel_row',min_value=2,max_value=8,step=1)
    hp_kernelcol = hp.Int('kernel_col',min_value=2,max_value=8,step=1)

    model.add(Conv2D(filters = hp_filters1, kernel_size = (hp_kernelrow,hp_kernelcol), activation = 'relu', input_shape = (IMG_SIZE, IMG_SIZE, 3)))
    model.add(MaxPooling2D(pool_size=(2,2)))

    hp_filters2 = hp.Int('filters2',min_value=32,max_value=128,step=8)
    model.add(Conv2D(filters = hp_filters2, kernel_size = (hp_kernelrow,hp_kernelcol), activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())

    hp_regularization = hp.Float('regularization',min_value=0.0,max_value=0.5,step=0.1)
    hp_units = hp.Int('units',min_value=64,max_value=512,step=8)
    model.add(Dense(units=hp_units, activation='relu', kernel_regularizer= keras.regularizers.l2(hp_regularization)))

    model.add(Dense(1,activation='sigmoid'))

    hp_learning_rate = hp.Choice('learning_rate',values=[1e-2,1e-3,1e-4])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),loss='binary_crossentropy',metrics=['accuracy'])

    return model

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.8,1.2]
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_dir = '/content/drive/MyDrive/DataSets/Breast Cancer/train'
val_dir = '/content/drive/MyDrive/DataSets/Breast Cancer/test'

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

tuner = kt.Hyperband(
    cnn_builder,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='/content/TunerLogs',
    project_name='Breast Cancer Classification'
)

tuner.search(train_generator, epochs=10, validation_data=val_generator)


Found 986 images belonging to 2 classes.
Found 395 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
120               |120               |filters1
6                 |6                 |kernel_row
3                 |3                 |kernel_col
56                |56                |filters2
0.2               |0.2               |regularization
392               |392               |units
0.01              |0.01              |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round



/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/2
 3/31 ━━━━━━━━━━━━━━━━━━━━ 6:18 14s/step - accuracy: 0.4375 - loss: 743.7997

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples //BATCH_SIZE,
    epochs=30,
    validation_data=val_generator,
    validation_steps=val_generator.samples //BATCH_SIZE,
    callbacks = [EarlyStopping(monitor='accuracy', patience=10, restore_best_weights=True)]
)

loss, accuracy = model.evaluate(val_generator, steps=val_generator.samples // BATCH_SIZE )
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)

keras.models.save_model(model,"BreastCancerClassifier.keras")

In [ ]:
# import shutil
# shutil.rmtree('/content/TunerLogs')